## Librerías

In [6]:
import pandas as pd
import sqlite3

## Instancia BBDD

In [7]:
# Conectamos con la base de datos chinook.db
connection = sqlite3.connect("datA/Bar_TB.DB")

# Obtenemos un cursor que utilizaremos para hacer las queries
crsr = connection.cursor()

## Función para interactuar con la BBDD

In [8]:
# Con esta función leemos los datos y lo pasamos a un DataFrame de Pandas
def sql_query(query):

    # Ejecuta la query
    crsr.execute(query)

    # Almacena los datos de la query 
    ans = crsr.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in crsr.description]

    return pd.DataFrame(ans,columns=names)

## Crear las tablas

### Ver si las tablas existen

In [9]:
res = crsr.execute("SELECT name FROM sqlite_master WHERE type='table'")
for name in res:
    print(name[0])

cerveza
bares
empleados
reparto


### Tabla Cerveza

In [14]:
## Cualquier 0 por delante de un número es de tipo string

query_cerveza = """ 
CREATE TABLE  cerveza (
    CodC VARCHAR(3),
    Envase VARCHAR(50),
    Capacidad FLOAT,
    Stock INT
)


"""
crsr.execute(query_cerveza)
# sql_query(query_cerveza) -- > NO FUNCIONA PORQUE ESTA FUNCIÓN EJECUTA Y SELECCIONA LO QUE HAY, PERO NO HAY NADA.

OperationalError: table cerveza already exists

### Tabla Bares

In [ ]:
query_bares = """ 
CREATE TABLE  bares (
    CodB VARCHAR(4),
    Nombre VARCHAR(50),
    Cif VARCHAR(10),
    Localidad VARCHAR(35)
)

"""
crsr.execute(query_bares)

### Tabla Empleados

In [ ]:
query_empleados = """ 
CREATE TABLE  empleados (
    CodE VARCHAR(2),
    Nombre VARCHAR(50),
    Sueldo INT
)

"""
crsr.execute(query_empleados)

### Tabla Reparto

In [ ]:
query_reparto = """ 
CREATE TABLE  reparto (
    CodE VARCHAR(2),
    CodB VARCHAR(4),
    CodC VARCHAR(3),
    Fecha DATE,
    Cantidad INT
)

"""
crsr.execute(query_reparto)

## Insertar datos

In [ ]:

query_insert = """ INSERT INTO cerveza VALUES
('01','Botella',0.2,3600),
('02','Botella',0.33,1200),
('03','Lata',0.33,2400),
('04','Botella',1,288),
('05','Barril',60,30)
"""
crsr.execute(query_insert)

In [ ]:
pd.read_sql('select * from cerveza', connection)

,CodC,Envase,Capacidad,Stock
0,01,Botella,0.20,3600
1,02,Botella,0.33,1200
2,03,Lata,0.33,2400
3,04,Botella,1.00,288
4,05,Barril,60.00,30


In [ ]:
query_insert2 = """ 
INSERT INTO reparto (CodE, CodB, CodC, Fecha, Cantidad) VALUES
('1', '001', '01', '2005-10-21', 240),
('1', '001', '02', '2005-10-21', 48),
('1', '002', '03', '2005-10-22', 60),
('1', '004', '05', '2005-10-22', 4),
('2', '002', '03', '2005-10-22', 48),
('2', '002', '05', '2005-10-23', 2),
('2', '004', '01', '2005-10-23', 480),
('2', '004', '02', '2005-10-24', 72),
('3', '003', '03', '2005-10-24', 48),
('3', '003', '04', '2005-10-25', 20)

"""
crsr.execute(query_insert2)

In [ ]:
## Datos en tabla Bares

bares = pd.DataFrame(
        {  
            'CodB' : ['001','002','003','004'],
            'Nombre' : [ 'Stop', 'Las Vegas', 'Club Social', 'Otra Ronca'],
            'Cif' : [ '11111111X','22222222Y',None,'33333333Z'],
            'Localidad' : ['Villa Botijo', 'Villa Botijo','Las Ranas', 'La esponja']
        }
)

bares.to_sql(name='bares', con=connection, if_exists="append",index=False)

4

In [ ]:
## Datos en tabla empleado

empleados = pd.DataFrame(
        {  
            'CodE' : ['1','2','3'],
            'Nombre' : [ 'Prudencio Caminero', 'Vicente Merario', 'Valentin Siempre'],
            'Sueldo' : [120000,110000,100000]
        }
)

empleados.to_sql(name='empleados', con=connection, if_exists="append",index=False)

3

In [ ]:
connection.commit()
connection.close()

## Respuestas

In [15]:
# Conectamos con la base de datos chinook.db
connection = sqlite3.connect("datA/Bar_TB.DB")

# Obtenemos un cursor que utilizaremos para hacer las queries
crsr = connection.cursor()

In [17]:
pd.read_sql('select * from reparto', connection)

,CodE,CodB,CodC,Fecha,Cantidad
0,1,001,01,2005-10-21,240
1,1,001,02,2005-10-21,48
2,1,002,03,2005-10-22,60
3,1,004,05,2005-10-22,4
4,2,002,03,2005-10-22,48
5,2,002,05,2005-10-23,2
6,2,004,01,2005-10-23,480
7,2,004,02,2005-10-24,72
8,3,003,03,2005-10-24,48
9,3,003,04,2005-10-25,20


In [25]:
answer1 = """ 
select DISTINCT emp.Nombre
from empleados as emp 
inner join reparto as re
on re.CodE = emp.CodE
inner join bares as ba
on ba.CodB = re.CodB
where ba.Nombre = "Stop"  and re.Fecha between '2005-10-17' and '2005-10-23'

"""
sql_query(answer1)

,Nombre
0,Prudencio Caminero
